# SIOPS 2022
> Neste notebook acessamos o catálogo de dados (datawarehouse) que cria o dataset do SIOPS 2022 e mostramos como ele é feito desde o acesso ao Banco de Dados da Aplicação SIOPS até a geração de arquivos Parquet com o dataset.
>

In [1]:
# import json
import duckdb
from fastduck import database
import ipywidgets
# from fastprogress.fastprogress import master_bar, progress_bar
import pandas as pd
# import requests
import os

Acessando o datawarehouse na pasta /data

In [2]:
db = duckdb.connect('../data/catalog.db') 

In [3]:
tables = (db.tables).df()
tables[tables['schema'].isin(['siops'])]

,catalog,schema,name,type,comment
244,catalog,siops,contas,VIEW,None
245,catalog,siops,despesas,VIEW,None
248,catalog,siops,lancamentos,VIEW,None
252,catalog,siops,receitas,VIEW,None
253,catalog,siops,receitas_deducoes,VIEW,None


## Construção

### Tabelas da Aplicação SIOPS Usadas

In [4]:
db.sql("select distinct table_name from duckdb_columns where schema_name = 'raw' and table_name like '%siops%'").df()

,table_name
0,siops__tb_municipio
1,siops__th_aut_transm
2,siopsuf__rl_proj_anoperiodoente_pasta
3,siopsuf__rl_proj_pasta_item
4,siops__tb_vl_valores
5,siopsuf__tb_proj_item
6,siopsuf__tb_vl_valores
7,siopsuf__tb_proj_pasta
8,siopsuf__rl_proj_anoperiodo_ente_fed
9,siopsuf__tb_proj_ente_federado


### TB_VL_VALORES

In [5]:
db.sql("""SELECT COUNT(*) FROM RAW.SIOPS__TB_VL_VALORES""")

count_star()
21879872


In [6]:
db.sql("""SELECT COUNT(*) FROM RAW.SIOPSUF__TB_VL_VALORES""")

count_star()
173590


In [7]:
21_879_872 + 173_590

22053462

### Unindo os dois bancos

In [8]:
todos_valores = db.sql("""
    WITH
    -- TODOS_VALORES une tb_vl_valores de estados e municípios
    TODOS_VALORES AS (
        SELECT
            CO_MUNICIPIO AS CODIGO_IBGE,
            * EXCLUDE (CO_MUNICIPIO)
        FROM
            RAW.SIOPS__TB_VL_VALORES
        UNION ALL
        SELECT
            CO_UF AS CODIGO_IBGE,
            * EXCLUDE (CO_UF)
        FROM
            RAW.SIOPSUF__TB_VL_VALORES
    ) SELECT * FROM TODOS_VALORES
                       """)
todos_valores.df().head(5)

,CODIGO_IBGE,CO_PASTA,CO_ITEM,CO_TIPO,NU_CNPJ_INSTITUICAO,NU_ANO,NU_PERIODO,CO_INSTITUICAO_CLIENTE,QT_PERIODOS,NU_VALOR,NU_ANO_PREENCHIDO,NU_PERIODO_PREENCHIDO,CO_GRUPO,CO_ITEM_EXIBICAO,CO_PASTA_HIERARQUIA
0,211120,1,1634,20,-1,2022,2,-1,1,29142627.73,2022,2,1,None,1
1,211120,1,1634,24,-1,2022,2,-1,1,29142627.73,2022,2,1,None,1
2,211120,1,1635,24,-1,2022,2,-1,1,33912670.99,2022,2,1,None,1
3,211120,1,1636,24,-1,2022,2,-1,1,21645338.90,2022,2,1,None,1
4,211120,1,1637,24,-1,2022,2,-1,1,8140699.52,2022,2,1,None,1


### Removendo as contas não operacionais (agregadoras)

A ideia do nosso dataset lancamentos é conter apenas dados inseridos pelos entes e remover dados derivados ainda que estejam salvos nos bancos

In [9]:
db.sql(""" 
            SELECT * FROM SIOPS.CONTAS
        """)

competencia,CODIGO_CONTA,DESCRICAO_CONTA,CODIGO_CONTA_SIOPS,TIPO_CONTA,ATIVO_ESTADO,ATIVO_DF,ATIVO_MUNICIPIO
2022,1.0.0.0.00.0.0,Receitas Correntes,1629,agregadora,S,S,S
2022,1.1.0.0.00.0.0,"Impostos, Taxas e Contribuições de Melhoria",1630,agregadora,S,S,S
2022,1.1.1.0.00.0.0,Impostos,1631,agregadora,S,S,S
...,...,...,...,...,...,...,...
2022,ACDO000007,Total das Despesas com Saúde Consideradas em ASPS,774,agregadora,S,S,S
2022,ACRO000001,TOTAL GERAL DAS RECEITAS,775,agregadora,S,S,S


In [10]:
operacionais = db.sql(""" 
       SELECT * FROM todos_valores WHERE CO_ITEM in 
        (
            SELECT CODIGO_CONTA_SIOPS::TEXT FROM SIOPS.CONTAS WHERE TIPO_CONTA = 'operacional'
        )""")
operacionais

CODIGO_IBGE,CO_PASTA,CO_ITEM,CO_TIPO,NU_CNPJ_INSTITUICAO,NU_ANO,NU_PERIODO,CO_INSTITUICAO_CLIENTE,QT_PERIODOS,NU_VALOR,NU_ANO_PREENCHIDO,NU_PERIODO_PREENCHIDO,CO_GRUPO,CO_ITEM_EXIBICAO,CO_PASTA_HIERARQUIA
430980,6,381,9,-1,2022,2,-1,1,147356.71,2022,2,1,None,6
430980,6,381,13,-1,2022,2,-1,1,268092.8,2022,2,1,None,6
430980,6,381,12,-1,2022,2,-1,1,151702.2,2022,2,1,None,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,5,642,12,-1,2022,2,-1,1,26012130.0,2022,2,1,None,5
29,5,642,14,-1,2022,2,-1,1,200499.9,2022,2,1,None,5


### Filtrando apenas dados de Receitas e Despesas
O banco do SIOPS é organizado de acordo como a interface gráfica funciona.  Portanto, precisamos dizer de que relatórios (PASTAS) e de que colunas (FASES entre outros conceitos) queremos valores.


In [11]:
db.sql("""SELECT * FROM RAW.SIOPSUF__TB_PROJ_COLUNA""")

CO_SEQ_COLUNA,NO_COLUNA,CO_COLUNA_INTERNO,NO_COMPLETO_COLUNA,DT_INCLUSAO,ST_UTILIZADO_DESCRICAO,TP_COLUNA
83,Restos a Pagar Inscritos não Processados,83,Restos a Pagar Incritos não Processados,NaN,NaN,VL
84,Restos a Pagar Inscritos Total,84,Restos a Pagar Incritos Total,NaN,NaN,VL
85,Restos a Pagar Pagos Até o Bimestre,85,Restos a Pagar Pagos Até o Bimestre,NaN,NaN,TL
...,...,...,...,...,...,...
81,Total Exercícios Anteriores,81,Total Exercícios Anteriores,NaN,NaN,VL
82,Restos a Pagar Inscritos Processados,82,Restos a Pagar Incritos Processados,NaN,NaN,VL


In [12]:
receitas_despesas = db.sql(""" SELECT * FROM operacionais 
            -- 3_1 3_2... 3_18, 4_1..4_18..., 95_1..95_18 são as pastas_hierarquias válidas
            WHERE (REGEXP_MATCHES(co_pasta_hierarquia, '^(3|4|6|7|8|9|10|86|87|88|89|90|94|95)_([1-9]|1[0-8])$') OR CO_PASTA = 1) 
            AND CO_TIPO < 23 -- Remove fases que são Totais Gerais
            AND CO_TIPO NOT IN (14, 20, 21) -- Remove Restos a Pagar e Totalizadoras de Receitas para base de cálculo ASPS
            """)
receitas_despesas

CODIGO_IBGE,CO_PASTA,CO_ITEM,CO_TIPO,NU_CNPJ_INSTITUICAO,NU_ANO,NU_PERIODO,CO_INSTITUICAO_CLIENTE,QT_PERIODOS,NU_VALOR,NU_ANO_PREENCHIDO,NU_PERIODO_PREENCHIDO,CO_GRUPO,CO_ITEM_EXIBICAO,CO_PASTA_HIERARQUIA
260580,12,386,9,-1,2022,2,-1,1,32922.38,2022,2,1,None,86_12
260580,12,386,10,-1,2022,2,-1,1,32922.38,2022,2,1,None,86_12
260580,12,386,13,-1,2022,2,-1,1,50000.0,2022,2,1,None,86_12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,13,463,10,-1,2022,2,-1,1,214336.0,2022,2,1,None,86_13
41,13,463,11,-1,2022,2,-1,1,214336.0,2022,2,1,None,86_13


### Filtrando dados homologados

Como o banco está reescrevendo por cima de valores já inseridos, checamos se a última atualização (cujos dados estão no banco) foram homologados e não incluímos no dataset caso não estejam.

In [13]:
homologados = db.sql("""
                     SELECT
            P.PERIODO AS COMPETENCIA,
            V.*
        FROM
            receitas_despesas V
            JOIN STAGING.SIOPS__PERIODOS P ON P.ANO = V.NU_ANO
            AND P.CODIGO_BIMESTRE_SIOPS = V.NU_PERIODO
            JOIN STAGING.SIOPS__HOMOLOGADOS H ON H.PERIODO = P.PERIODO
            AND H.IBGE_ENTE = V.CODIGO_IBGE
                     """)
homologados

COMPETENCIA,CODIGO_IBGE,CO_PASTA,CO_ITEM,CO_TIPO,NU_CNPJ_INSTITUICAO,NU_ANO,NU_PERIODO,CO_INSTITUICAO_CLIENTE,QT_PERIODOS,NU_VALOR,NU_ANO_PREENCHIDO,NU_PERIODO_PREENCHIDO,CO_GRUPO,CO_ITEM_EXIBICAO,CO_PASTA_HIERARQUIA
2022-6,261260,1,1634,15,-1,2022,2,-1,1,5250501.09,2022,2,1,None,1
2022-6,261260,1,1634,17,-1,2022,2,-1,1,5250501.09,2022,2,1,None,1
2022-6,261260,1,1634,18,-1,2022,2,-1,1,3079283.99,2022,2,1,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-6,41,13,463,10,-1,2022,2,-1,1,214336.0,2022,2,1,None,86_13
2022-6,41,13,463,11,-1,2022,2,-1,1,214336.0,2022,2,1,None,86_13


### Enriquecendo os dados


In [14]:
final = db.sql(""" 
                SELECT
     H.COMPETENCIA as competencia, -- mudar para periodo?  exercicio?
     H.CODIGO_IBGE as ibge,
     S.ENTE as ente,
     CASE WHEN S.CAPITAL = 1 THEN 'S' ELSE 'N' END AS capital,
     S.REGIAO as regiao,
     S.UF as uf,
     CASE
         WHEN S.ESFERA = 'D' THEN 'Distrital'
         WHEN S.ESFERA = 'M' THEN 'Municipal'
         WHEN S.ESFERA = 'E' THEN 'Estadual'
         WHEN S.ESFERA = 'U' THEN 'Federal'
     END AS esfera,
     S.POPULACAO as populacao,
     -- remove conteúdo entre parenteses e espaços em branco
     TRIM(REGEXP_REPLACE(REGEXP_REPLACE(C.NO_COLUNA, '\s*=.*$', ''), '\s*\([a-z]\)', '')) AS fase,
    -- C.NO_COLUNA as fase,
     FS.FONTE as fonte,
     FS.SUBFUNCAO AS destinacao,
     CT.CODIGO_CONTA AS conta,
     CT.DESCRICAO_CONTA AS descricao_conta,
     H.NU_VALOR AS valor_nominal
 FROM
     homologados H 
     JOIN SIOPS.CONTAS CT ON CT.CODIGO_CONTA_SIOPS = H.CO_ITEM
     JOIN RAW.SIOPSUF__TB_PROJ_COLUNA C ON H.CO_TIPO = C.CO_SEQ_COLUNA
     LEFT JOIN STAGING.SIOPS__FONTES_SUBFUNCOES FS ON FS.PASTA_HIERARQUIA = H.CO_PASTA_HIERARQUIA
     JOIN SICONFI.ENTES S ON H.CODIGO_IBGE = S.codigo_ibge_6
 ORDER BY competencia, ibge, fase, conta, fonte, destinacao;
               """)

In [15]:
final

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

competencia,ibge,ente,capital,regiao,uf,esfera,populacao,fase,fonte,destinacao,conta,descricao_conta,valor_nominal
2022-6,11,Rondônia,N,N,BR,Estadual,1616379,Dedução Para Formação do FUNDEB,None,None,1.1.1.8.01.2.1,Imposto sobre a Propriedade de Veículos Automotores - Principal,57257944.51
2022-6,11,Rondônia,N,N,BR,Estadual,1616379,Dedução Para Formação do FUNDEB,None,None,1.1.1.8.02.1.1,Imposto sobre Operações Relativas à Circulação de Mercadorias e sobre Prestações de Serviços de Transporte Interestadual e Intermunicipal e de Comunicação - Principal,1776587706.65
2022-6,11,Rondônia,N,N,BR,Estadual,1616379,Dedução Para Formação do FUNDEB,None,None,1.7.1.8.01.6.0,Cota-Parte do Imposto Sobre Produtos Industrializados – Estados Exportadores de Produtos Industrializados,3734168.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-6,522230,Vila Propício,N,CO,GO,Municipal,5646,Receitas Realizadas Brutas,None,None,2.4.2.8.10.1.0,Transferências de Convênios dos Estados para o Sistema Único de Saúde – SUS,150000.0
2022-6,522230,Vila Propício,N,CO,GO,Municipal,5646,Receitas Realizadas Brutas,None,None,2.4.4.8.01.9.0,Outras Transferências de Convênios de Instituições Privadas,55758.54


In [16]:
dataset = db.sql("SELECT * FROM siops.lancamentos").df()
dataset

,competencia,ibge,ente,capital,regiao,uf,esfera,populacao,fase,fonte,destinacao,conta,descricao_conta,valor_nominal
0,2022-6,11,Rondônia,N,N,BR,Estadual,1616379,Dedução Para Formação do FUNDEB,None,None,1.1.1.8.01.2.1,Imposto sobre a Propriedade de Veículos Automo...,5.725794e+07
1,2022-6,11,Rondônia,N,N,BR,Estadual,1616379,Dedução Para Formação do FUNDEB,None,None,1.1.1.8.02.1.1,Imposto sobre Operações Relativas à Circulação...,1.776588e+09
2,2022-6,11,Rondônia,N,N,BR,Estadual,1616379,Dedução Para Formação do FUNDEB,None,None,1.7.1.8.01.6.0,Cota-Parte do Imposto Sobre Produtos Industria...,3.734168e+06
3,2022-6,11,Rondônia,N,N,BR,Estadual,1616379,Dedução de Transferências Const. e Legais a Mu...,None,None,1.1.1.8.01.2.1,Imposto sobre a Propriedade de Veículos Automo...,2.528905e+08
4,2022-6,11,Rondônia,N,N,BR,Estadual,1616379,Dedução de Transferências Const. e Legais a Mu...,None,None,1.1.1.8.01.2.1,Imposto sobre a Propriedade de Veículos Automo...,2.639310e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3777575,2022-6,522230,Vila Propício,N,CO,GO,Municipal,5646,Receitas Realizadas Brutas,None,None,1.9.2.2.00.0.0,Restituições,1.740471e+04
3777576,2022-6,522230,Vila Propício,N,CO,GO,Municipal,5646,Receitas Realizadas Brutas,None,None,2.4.1.8.10.2.0,Transferências de Convênio da União destinadas...,1.999400e+05
3777577,2022-6,522230,Vila Propício,N,CO,GO,Municipal,5646,Receitas Realizadas Brutas,None,None,2.4.1.8.10.9.0,Outras Transferências de Convênios da União,1.051306e+06
3777578,2022-6,522230,Vila Propício,N,CO,GO,Municipal,5646,Receitas Realizadas Brutas,None,None,2.4.2.8.10.1.0,Transferências de Convênios dos Estados para o...,1.500000e+05


In [17]:
receitas = db.sql("SELECT * FROM siops.receitas").df()
receitas

,competencia,ibge,ente,capital,regiao,uf,esfera,populacao,conta,descricao_conta,receita_realizada
0,2022-6,11,Rondônia,N,N,BR,Estadual,1616379,1.1.1.3.00.0.0,Impostos sobre a Renda e Proventos de Qualquer...,6.208765e+08
1,2022-6,11,Rondônia,N,N,BR,Estadual,1616379,1.1.1.8.01.2.1,Imposto sobre a Propriedade de Veículos Automo...,4.643323e+08
2,2022-6,11,Rondônia,N,N,BR,Estadual,1616379,1.1.1.8.01.2.2,Imposto sobre a Propriedade de Veículos Automo...,2.064342e+07
3,2022-6,11,Rondônia,N,N,BR,Estadual,1616379,1.1.1.8.01.2.3,Imposto sobre a Propriedade de Veículos Automo...,1.769272e+07
4,2022-6,11,Rondônia,N,N,BR,Estadual,1616379,1.1.1.8.01.2.4,Imposto sobre a Propriedade de Veículos Automo...,1.992452e+06
...,...,...,...,...,...,...,...,...,...,...,...
260889,2022-6,522230,Vila Propício,N,CO,GO,Municipal,5646,1.9.2.2.00.0.0,Restituições,1.740471e+04
260890,2022-6,522230,Vila Propício,N,CO,GO,Municipal,5646,2.4.1.8.10.2.0,Transferências de Convênio da União destinadas...,1.999400e+05
260891,2022-6,522230,Vila Propício,N,CO,GO,Municipal,5646,2.4.1.8.10.9.0,Outras Transferências de Convênios da União,1.051306e+06
260892,2022-6,522230,Vila Propício,N,CO,GO,Municipal,5646,2.4.2.8.10.1.0,Transferências de Convênios dos Estados para o...,1.500000e+05


In [18]:
despesas = db.sql("SELECT * FROM siops.despesas").df()
despesas

,competencia,ibge,ente,capital,regiao,uf,esfera,populacao,conta,descricao_conta,fonte,destinacao,despesa_empenhada,despesa_liquidada,saldo_empenho,despesa_paga,saldo_liquidado
0,2022-6,11,Rondônia,N,N,BR,Estadual,1616379,3.1.71.00.00.00,Transferências a Consórcios Públicos mediante ...,Transf. Fed. SUS - Manutenção,Suporte Profilático Terapêutico (303),1828800.09,1828800.09,0.00,1828800.09,0.00
1,2022-6,11,Rondônia,N,N,BR,Estadual,1616379,3.1.90.07.00.00,Contribuição a Entidades Fechadas de Previdência,Receitas Impostos e Transf.,Administrativas,134220.45,134220.45,0.00,134220.45,0.00
2,2022-6,11,Rondônia,N,N,BR,Estadual,1616379,3.1.90.12.00.00,Vencimentos e Vantagens Fixas - Pessoal Militar,Receitas Impostos e Transf.,Administrativas,534275.14,534275.14,0.00,534275.14,0.00
3,2022-6,11,Rondônia,N,N,BR,Estadual,1616379,3.1.90.13.00.00,Obrigações Patronais,Receitas Impostos e Transf.,Administrativas,2638985.02,2633518.59,5466.43,2613974.56,19544.03
4,2022-6,11,Rondônia,N,N,BR,Estadual,1616379,3.1.90.13.00.00,Obrigações Patronais,Transf. Fed. SUS - Manutenção,Administrativas,17666.04,6021.54,11644.50,6021.54,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465581,2022-6,522230,Vila Propício,N,CO,GO,Municipal,5646,4.4.90.51.00.00,Obras e Instalações,Transf. Estadual SUS,Atenção Básica (301),160045.31,NaN,NaN,NaN,NaN
465582,2022-6,522230,Vila Propício,N,CO,GO,Municipal,5646,4.4.90.51.00.00,Obras e Instalações,Receitas Impostos e Transf.,Atenção Básica (301),1080321.94,NaN,NaN,NaN,NaN
465583,2022-6,522230,Vila Propício,N,CO,GO,Municipal,5646,4.4.90.52.08.00,"Aparelhos, Equipamentos, Utensílios Médico Odo...",Receitas Impostos e Transf.,Atenção Básica (301),90425.00,90425.00,0.00,8425.00,82000.00
465584,2022-6,522230,Vila Propício,N,CO,GO,Municipal,5646,4.4.90.52.99.00,Outros Equipamentos e Material Permanente,Receitas Impostos e Transf.,Atenção Básica (301),353953.00,352453.00,1500.00,352453.00,0.00


In [20]:
db.sql("SELECT * FROM siops.receitas_deducoes").df()

,competencia,ibge,ente,capital,regiao,uf,esfera,populacao,conta,descricao_conta,receita_realizada_bruta,deducoes_transf,deducoes_fundeb,outras_deducoes
0,2022-6,11,Rondônia,N,N,BR,Estadual,1616379,1.1.1.3.00.0.0,Impostos sobre a Renda e Proventos de Qualquer...,6.208765e+08,NaN,NaN,NaN
1,2022-6,11,Rondônia,N,N,BR,Estadual,1616379,1.1.1.8.01.2.1,Imposto sobre a Propriedade de Veículos Automo...,4.643323e+08,707300402.2,57257944.51,NaN
2,2022-6,11,Rondônia,N,N,BR,Estadual,1616379,1.1.1.8.01.2.2,Imposto sobre a Propriedade de Veículos Automo...,2.064342e+07,NaN,NaN,NaN
3,2022-6,11,Rondônia,N,N,BR,Estadual,1616379,1.1.1.8.01.2.3,Imposto sobre a Propriedade de Veículos Automo...,1.769272e+07,NaN,NaN,NaN
4,2022-6,11,Rondônia,N,N,BR,Estadual,1616379,1.1.1.8.01.2.4,Imposto sobre a Propriedade de Veículos Automo...,1.992452e+06,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261363,2022-6,522230,Vila Propício,N,CO,GO,Municipal,5646,1.9.2.2.00.0.0,Restituições,1.740471e+04,NaN,NaN,NaN
261364,2022-6,522230,Vila Propício,N,CO,GO,Municipal,5646,2.4.1.8.10.2.0,Transferências de Convênio da União destinadas...,1.999400e+05,NaN,NaN,NaN
261365,2022-6,522230,Vila Propício,N,CO,GO,Municipal,5646,2.4.1.8.10.9.0,Outras Transferências de Convênios da União,1.051306e+06,NaN,NaN,NaN
261366,2022-6,522230,Vila Propício,N,CO,GO,Municipal,5646,2.4.2.8.10.1.0,Transferências de Convênios dos Estados para o...,1.500000e+05,NaN,NaN,NaN


In [21]:
import os
os.chdir('..')

In [22]:
db.close()

In [23]:
!sqlmesh audit | grep -E "(PASS|FAIL)"

number_of_rows on model siops.lancamentos ✅ PASS.
not_null on model siops.lancamentos ✅ PASS.
accepted_values on model siops.lancamentos ✅ PASS.
accepted_range on model siops.lancamentos ❌ FAIL [1764].
todos_municipios_tem_receita_iptu on model siops.receitas ❌ FAIL [27].
todos_municipios_tem_receita_itbi on model siops.receitas ❌ FAIL [36].
todos_municipios_tem_receita_iss on model siops.receitas ❌ FAIL [2].
todos_municipios_tem_receita_cotaparteipva on model siops.receitas ❌ FAIL [1].
todos_estados_tem_receita_ipva_acima_minimo on model siops.receitas ✅ PASS.
todos_estados_tem_receita_itcmd_acima_minimo on model siops.receitas ✅ PASS.
saldo_positivo on model siops.despesas ❌ FAIL [2].
